In [15]:
import dspy
from data_models.dspy.dspy_o_function_summary import DspyFunctionSummary
from data_models.dspy.dspy_unoplat_fs_function_subset import DspyUnoplatFunctionSubset
from data_models.dspy.dspy_unoplat_fs_node_subset import DspyUnoplatNodeSubset


In [ ]:

together = dspy.Together(model="mistralai/Mistral-7B-v0.1")

In [19]:
class CodeConfluenceFunctionSignature(dspy.Signature):
    """This signature contains class metadata and function metadata with function content and returns descriptive function summary"""
    dspy_class_subset: DspyUnoplatNodeSubset = dspy.InputField(desc="This will contain class metadata")
    dspy_function_subset: DspyUnoplatFunctionSubset = dspy.InputField(desc="This will contain function metadata with function content")
    dspy_function_summary: DspyFunctionSummary = dspy.OutputField(desc="This will contain function summary")



In [20]:
class CodeConfluenceFunctionSummaryOptimiserSignature(dspy.Signature):
    """This signature contains function summary and returns optimised and accurate function summary"""
    dspy_function_input_summary: DspyFunctionSummary = dspy.InputField(desc="This will contain function summary")
    dspy_function_output_summary: DspyFunctionSummary = dspy.OutputField(desc="This will contain optimised function summary")


In [21]:
class CodeConfluenceFunctionModule(dspy.Module):
    def __init__(self):
        super().__init__()
        self.generate_function_summary = dspy.TypedChainOfThought(CodeConfluenceFunctionSignature)
        self.optimise_function_summary = dspy.TypedChainOfThought(CodeConfluenceFunctionSummaryOptimiserSignature)

    def forward(self, function_metadata, class_metadata):
        function_summary = self.generate_function_summary( dspy_class_subset = class_metadata, dspy_function_subset= function_metadata)
        optimised_function_summary = self.optimise_function_summary(function_summary)
        print(optimised_function_summary.dspy_function_output_summary)
        return optimised_function_summary

In [18]:
import json
from data_models.chapi_unoplat_node import Node
from data_models.dspy.dspy_unoplat_fs_annotation_subset import DspyUnoplatAnnotationSubset
from data_models.dspy.dspy_unoplat_fs_function_call_subset import DspyUnoplatFunctionCallSubset
from data_models.dspy.dspy_unoplat_fs_function_subset import DspyUnoplatFunctionSubset
from data_models.dspy.dspy_unoplat_fs_node_subset import DspyUnoplatNodeSubset

try:
    with open('springstarterjava1_codes.json', 'r') as file:
        springstarterjava1_codes = json.load(file)
except FileNotFoundError:
    print("Error: File 'springstarterjava1_codes.json' not found.")
    springstarterjava1_codes = []
except json.JSONDecodeError:
    print("Error: File 'springstarterjava1_codes.json' contains invalid JSON.")
    springstarterjava1_codes = []

node_subsets = []

function_subsets = []

for item in springstarterjava1_codes:
    try:
        node = Node(**item)
        print("node name",node.node_name)
        node_subset = DspyUnoplatNodeSubset(
            NodeName=node.node_name,
            Imports=node.imports,
            Extend=node.extend,
            MultipleExtend=node.multiple_extend,
            Fields=node.fields,
            Annotations=[DspyUnoplatAnnotationSubset(**annotation.model_dump()) for annotation in node.annotations]
        )
        
        
        for func in node.functions:
            function_subset = DspyUnoplatFunctionSubset(
                Name=func.name,
                ReturnType=func.return_type,
                Annotations=func.annotations,
                LocalVariables=func.local_variables,
                Content=func.content,
                FunctionCalls=[DspyUnoplatFunctionCallSubset(NodeName=call.node_name, FunctionName=call.function_name, Parameters=call.parameters) for call in func.function_calls]
            )
            


        function_subsets.append(function_subset)
    except AttributeError as e:
        print(f"Error processing node data: {e}")

# Optionally, print or process the subsets
print(node_subsets)
print(function_subsets)

node name Order
Order
{'node_name': 'Order', 'type': 'CLASS', 'file_path': 'src/main/java/com/datastax/examples/order/Order.java', 'module': 'root', 'package': 'com.datastax.examples.order', 'multiple_extend': None, 'fields': [{'type_type': 'OrderPrimaryKey', 'type_key': 'key', 'annotations': [{'name': 'PrimaryKey', 'key_values': [], 'position': {'start_line': 13, 'start_line_position': 4, 'stop_line': 13, 'stop_line_position': 5}}]}, {'type_type': 'Integer', 'type_key': 'productQuantity', 'annotations': [{'name': 'Column', 'key_values': [{'key': '"product_quantity"', 'value': '"product_quantity"'}], 'position': {'start_line': 16, 'start_line_position': 4, 'stop_line': 16, 'stop_line_position': 30}}, {'name': 'CassandraType', 'key_values': [{'key': 'type', 'value': 'CassandraType.Name.INT'}], 'position': {'start_line': 17, 'start_line_position': 4, 'stop_line': 17, 'stop_line_position': 48}}]}, {'type_type': 'String', 'type_key': 'productName', 'annotations': [{'name': 'Column', 'key_v

In [6]:
node_subsets[0]

DspyUnoplatNodeSubset(node_name=None, multiple_extend=None, fields=[], extend=None, imports=[], annotations=[])

In [1]:
import dspy
import os
from data_models.chapi_unoplat_function import Function
from data_models.chapi_unoplat_function_summary import FunctionSummary
from loader import JsonLoader
from loader import JsonParser

ollama_mistral = dspy.OllamaLocal(model='mistral:7b-instruct-fp16')
dspy.configure(lm=ollama_mistral)


/Users/jayghiya/Documents/unoplat/unoplat-codebase-understanding/unoplat-code-confluence/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'data_models.chapi_unoplat_function_summary'

In [8]:
class UnoplatFunctionSummary(dspy.Signature):
    """Generate accurate Summary only based on user metadata about function"""
    
    input: Function = dspy.InputField(desc="will contain all relevant function metadata")
    output: FunctionSummary = dspy.OutputField(desc="summarisation of function")


In [10]:
from typing import List
from data_models.chapi_unoplat_class_summary import ClassSummary
from data_models.chapi_unoplat_node import Node


class UnoplatClassSummary(dspy.Signature):
    """Generate accurate summary only based on List of Function summaries and class metadata"""
    input: List[FunctionSummary] = dspy.InputField(desc="This contains list of function summaries that needs to be understood in a step by step fashion.")
    context: Node = dspy.InputField(desc="This will contain class metadata." )
    output: ClassSummary = dspy.OutputField(desc="This should contain class summary based on user shared metadata.")

In [11]:
from typing import List

class UnoplatModuleFunctionSummary(dspy.Module):
    """Generate summary for all functions of a class"""
    def __init__(self):
        super().__init__()
        self.generate_function_summary = dspy.TypedChainOfThought(UnoplatFunctionSummary)
        

    def forward(self, list_function = None) -> str:
        function_summaries_context = []
        
        for item in list_function:
            function_summaries_context.append(self.generate_function_summary(item))

        
        return function_summaries_context    


            
        

In [13]:
from dspy.teleprompt import BootstrapFewShot
# Set up the teleprompter
teleprompter = BootstrapFewShot()

# Compile the program without a training dataset
compiled_summary = teleprompter.compile(UnoplatModuleClassSummary(),trainset=None)

#cot_predictor = dspy.TypedChainOfThought(UnoplatFunctionSummary)

TypeError: object of type 'NoneType' has no len()

In [4]:
iload_json = JsonLoader()

iparse_json = JsonParser()


meadata_classes = iload_json.load_json_from_file("springstarterjava1_codes.json")

package_dict = iparse_json.parse_json_to_nodes(meadata_classes,None)

print(package_dict.keys)

function: Function = package_dict['com.datastax.examples.order'][2].functions[0]
print(function)

prediction = cot_predictor(input=function,options={"format": "json"})

ollama_mistral.inspect_history(n=1)

2024-06-12 11:02:15.265 | INFO     | loader.json_loader:load_json_from_file:28 - JSON data successfully loaded from springstarterjava1_codes.json


<built-in method keys of dict object at 0x1761ce180>
name='root' return_type='ModelAndView' function_calls=[] annotations=[Annotation(name='RequestMapping', position=Position(start_line=17, start_line_position=4, stop_line=17, stop_line_position=59))] position=Position(start_line=18, start_line_position=11, stop_line=20, stop_line_position=4) local_variables=[FieldModel(type_type='ModelAndView', type_value='return', type_key=None)] body_hash=763749358 content='ModelAndView root() {        return new ModelAndView("redirect:/swagger-ui/");    '



Generate Summary in json based on FunctionSummary

---

Follow the following format.

Input: will contain all relevant function metadata

Past Error in Output: An error to avoid in the future

Reasoning: Let's think step by step in order to ${produce the output}. We ...

Output:
summarisation of function. Respond with a single JSON object. 
You MUST use this format: {
"Summary": null,
"ImplementationSummary": null
}
JSON Schema: {"properties": 

'\n\n\nGenerate Summary in json based on FunctionSummary\n\n---\n\nFollow the following format.\n\nInput: will contain all relevant function metadata\n\nPast Error in Output: An error to avoid in the future\n\nReasoning: Let\'s think step by step in order to ${produce the output}. We ...\n\nOutput:\nsummarisation of function. Respond with a single JSON object. \nYou MUST use this format: {\n"Summary": null,\n"ImplementationSummary": null\n}\nJSON Schema: {"properties": {"Summary": {"anyOf": [{"type": "string"}, {"type": "null"}], "default": null, "description": "This should include high level summary of what function does based on function content and function metadata.", "title": "Summary"}, "ImplementationSummary": {"anyOf": [{"type": "string"}, {"type": "null"}], "default": null, "description": "This should include implementation details of the function in a step by step fashion with precise functional arguments and fields used to perform the operation. use all metadata shared for t

In [5]:
print(prediction)

Prediction(
    reasoning='1. Parse the input JSON string into a Python dictionary using the `json.loads()` function.\n2. Extract the relevant metadata from the dictionary, including the function name, return type, function calls, annotations, position, local variables, body hash, and content.\n3. Generate a summary of the function based on its name and return type. For example, "root() returns a ModelAndView object."\n4. Generate an implementation summary of the function by parsing the content string and extracting the relevant information. This may involve using regular expressions or other string manipulation techniques to identify the functional arguments and fields used in the operation.\n5. Combine the summary and implementation summary into a single JSON',
    output=FunctionSummary(summary='root() returns a ModelAndView object.', implementation_summary='The function takes no arguments and returns a new ModelAndView object with a redirect URL of /swagger-ui/')
)
